In [1]:
import os,csv
import time
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import argparse
import sys
import torchvision.models as models
from torchvision.models.resnet import ResNet, BasicBlock
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from PIL import Image

In [2]:
TRAIN_CSV_PATH = "/kaggle/input/morph/Dataset/Index/Train.csv"
VALID_CSV_PATH = "/kaggle/input/morph/Dataset/Index/Validation.csv"
TEST_CSV_PATH = "/kaggle/input/morph/Dataset/Index/Test.csv"
IMAGE_PATH = "/kaggle/input/morph/Dataset/Images"
# Argparse helper

parser = argparse.ArgumentParser()
parser.add_argument('--cuda',
                    type=int,
                    default=0)

parser.add_argument('--seed',
                    type=int,
                    default=0)

parser.add_argument('--numworkers',
                    type=int,
                    default=2)


parser.add_argument('--outpath',
                    type=str,
                    required=False,
                    default='./MORPH-OR')

parser.add_argument('--imp_weight',
                    type=int,
                    default=0)

args = parser.parse_args(args=[])

NUM_WORKERS = args.numworkers

if args.cuda >= 0:
    DEVICE = torch.device("cuda:%d" % args.cuda)
else:
    DEVICE = torch.device("cpu")

if args.seed == -1:
    RANDOM_SEED = None
else:
    RANDOM_SEED = args.seed

IMP_WEIGHT = args.imp_weight

Path = args.outpath
#print(Path)
if not os.path.exists(Path):
  os.mkdir(Path)
LOGFILE = os.path.join(Path, 'Training.log')
TEST_PREDICTIONS1 = os.path.join(Path, 'Age.log')
TEST_PREDICTIONS2 = os.path.join(Path, 'Gender.log')
TEST_ALLPROBAS = os.path.join(Path, 'Test_allprobas.tensor')

# Logging

header = []

header.append('PyTorch Version: %s' % torch.__version__)
header.append('CUDA device available: %s' % torch.cuda.is_available())
header.append('Using CUDA device: %s' % DEVICE)
header.append('Random Seed: %s' % RANDOM_SEED)
header.append('Task Importance Weight: %s' % IMP_WEIGHT)
header.append('Output Path: %s' % Path)
header.append('Script: %s' % sys.argv[0])

with open(LOGFILE, 'w') as f:
    for entry in header:
        print(entry)
        f.write('%s\n' % entry)
        f.flush()

PyTorch Version: 1.13.0
CUDA device available: True
Using CUDA device: cuda:0
Random Seed: 0
Task Importance Weight: 0
Output Path: ./MORPH-OR
Script: /opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py


In [3]:
##########################
# SETTINGS
##########################

# Hyperparameters
learning_rate = 0.0005
num_epochs = 75

# Architecture
BATCH_SIZE = 256
GRAYSCALE = False

df = pd.read_csv(TRAIN_CSV_PATH)
ages = df['age'].values
gender = df['gender'].values
del df
ages = torch.tensor(ages, dtype=torch.float)
gender = torch.tensor(gender, dtype=torch.float)

def task_importance_weights(label_array):
    uniq = torch.unique(label_array)
    num_examples = label_array.size(0)

    m = torch.zeros(uniq.shape[0])

    for i, t in enumerate(torch.arange(torch.min(uniq), torch.max(uniq))):
        m_k = torch.max(torch.tensor([label_array[label_array > t].size(0), 
                                      num_examples - label_array[label_array > t].size(0)]))
        m[i] = torch.sqrt(m_k.float())

    imp = m/torch.max(m)
    return imp


# Data-specific scheme
if not IMP_WEIGHT:
    imp = torch.ones(62-1, dtype=torch.float)
elif IMP_WEIGHT == 1:
    imp = task_importance_weights(ages)
    imp = imp[0:62-1]
else:
    raise ValueError('Incorrect importance weight parameter.')
imp = imp.to(DEVICE)

In [4]:
class MORPH(Dataset):
    def __init__(self,
                 csv_path, img_dir, transform=None):

        df = pd.read_csv(csv_path,)
        self.img_dir = img_dir
        self.csv_path = csv_path
        self.img_names =df["filepath"].values
        self.y = df["age"].values
        self.z = df["gender"].values
        self.transform = transform

    def __getitem__(self, index):
        img = Image.open(os.path.join(self.img_dir,
                                      self.img_names[index]))

        if self.transform is not None:
            img = self.transform(img)
        label1 = self.y[index]
        label2 = self.z[index]
        levels = [1]*label1 + [0]*(62 - 1 - label1)
        levels = torch.tensor(levels, dtype=torch.float32)

        return img, label1, label2, levels
    
    def __len__(self):
        return self.y.shape[0]


custom_transform = transforms.Compose([transforms.Resize((128, 128)),
                                       transforms.RandomCrop((120, 120)),
                                       transforms.ToTensor()])

train_dataset = MORPH(csv_path=TRAIN_CSV_PATH,
                              img_dir=os.path.join(IMAGE_PATH,"Train"),
                              transform=custom_transform)

custom_transform2 = transforms.Compose([transforms.Resize((128, 128)),
                                       transforms.CenterCrop((120, 120)),
                                       transforms.ToTensor()])

test_dataset =MORPH(csv_path=TEST_CSV_PATH,
                             img_dir=os.path.join(IMAGE_PATH,"Test"),
                             transform=custom_transform2)

valid_dataset = MORPH(csv_path=VALID_CSV_PATH,
                              img_dir=os.path.join(IMAGE_PATH,"Validation"),
                              transform=custom_transform2)

train_loader = DataLoader(dataset=train_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=True,
                          num_workers=NUM_WORKERS)

valid_loader = DataLoader(dataset=valid_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=False,
                          num_workers=NUM_WORKERS)

test_loader = DataLoader(dataset=test_dataset,
                         batch_size=BATCH_SIZE,
                         shuffle=False,
                         num_workers=NUM_WORKERS)

In [5]:
##########################
# MODEL
##########################
class resnet(ResNet):
    def __init__(self, block, layers, num_classes, grayscale):
        self.num_classes=num_classes
        self.block=BasicBlock
        self.inplanes = 64
        if grayscale:
            in_dim = 1
        else:
            in_dim = 3
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(in_dim, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AvgPool2d(4)
        self.fc1 = nn.Linear(512,(self.num_classes-1)*2, bias=True)
        self.fc2 = nn.Linear(512, 1, bias=True)
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, (2. / n)**.5)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        logits1 = self.fc1(x)
        logits2 = self.fc2(x)
        logits1 = logits1.view(-1, (self.num_classes-1), 2)
        probas1 = F.softmax(logits1, dim=2)[:, :, 1]
        probas2 = torch.sigmoid(logits2)
        return logits1, logits2, probas1, probas2


def resnet34(num_classes, grayscale):
    model=resnet(block=BasicBlock,
                   layers=[3, 4, 6, 3],
                   num_classes=num_classes,
                   grayscale=grayscale)
    return model

In [6]:
###########################################
# Initialize Cost, Model, and Optimizer
###########################################
def cost_fn(logits, levels, imp):
    val = (-torch.sum((F.log_softmax(logits, dim=2)[:, :, 1]*levels
                      + F.log_softmax(logits, dim=2)[:, :, 0]*(1-levels))*imp, dim=1))
    return torch.mean(val)


torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed(RANDOM_SEED)
model = resnet34(num_classes=62,grayscale=GRAYSCALE)
model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 


def compute_mae_and_mse(model, data_loader, device):
    mae, mse, num_examples1, num_examples2, correct_pred = 0, 0, 0, 0, 0
    for i, (features, target1, target2, levels) in enumerate(data_loader):

        features = features.to(device)
        target1 = target1.to(device)
        target2 = target2.float().to(device)
        logits1,logits2,probas1,probas2 = model(features)
        
        predict_levels = probas1 > 0.5
        predictions = probas2 > 0.5
        predicted_labels1 = torch.sum(predict_levels, dim=1)
        predicted_labels2 = torch.sum(predictions, dim=1)
        num_examples1 += target1.size(0)
        num_examples2 += target2.size(0)
        
        mae += torch.sum(torch.abs(predicted_labels1 - target1))
        mse += torch.sum((predicted_labels1 - target1)**2)
        correct_pred += (predicted_labels2==target2).sum()
    
    mae = mae.float() / num_examples1
    mse = mse.float() / num_examples1
    gender_acc=correct_pred.float()/num_examples2*100
    return mae, mse, gender_acc


start_time = time.time()

best_mae, best_rmse, best_epoch = 999, 999, -1
for epoch in range(num_epochs):

    model.train()
    for batch_idx, (features, target1, target2, levels) in enumerate(train_loader):
        features = features.to(DEVICE)
        target1 = target1.to(DEVICE)
        target2 = target2.float().to(DEVICE)
        levels = levels.to(DEVICE)

        # FORWARD AND BACK PROP
        logits1, logits2, probas1,probas2 = model(features)
        cost1 = cost_fn(logits1,levels,imp)
        demo = nn.BCELoss()
        act = nn.Sigmoid()
        cost2 = demo(act(logits2),target2.unsqueeze(1))
        
        cost=cost1+cost2
        optimizer.zero_grad()

        cost.backward()

        # UPDATE MODEL PARAMETERS
        optimizer.step()

        # LOGGING
        if not batch_idx % 20:
            s = ('Epoch: %03d/%03d | Batch %04d/%04d | Cost: %.4f'
                 % (epoch+1, num_epochs, batch_idx,
                     len(train_dataset)//BATCH_SIZE, cost))
            print(s)
            with open(LOGFILE, 'a') as f:
                f.write('%s\n' % s)

    model.eval()
    with torch.set_grad_enabled(False):
        valid_mae, valid_mse, valid_gender = compute_mae_and_mse(model, valid_loader,
                                                   device=DEVICE)

    if valid_mae < best_mae:
        best_mae, best_rmse, best_gacc, best_epoch = valid_mae, torch.sqrt(valid_mse),valid_gender, epoch
        ########## SAVE MODEL #############
        torch.save(model.state_dict(), os.path.join(Path, 'Age_and_Gender.pt'))

    s='MAE/RMSE/Gender: | Current Valid: %.2f/%.2f/%.2f Ep. %d | Best Valid : %.2f/%.2f/%.2f Ep. %d' % (
        valid_mae, torch.sqrt(valid_mse), valid_gender, epoch, best_mae, best_rmse, best_gacc, best_epoch)
    print(s)
    with open(LOGFILE, 'a') as f:
        f.write('%s\n' % s)

    s = 'Time elapsed: %.2f min' % ((time.time() - start_time)/60)
    print(s)
    with open(LOGFILE, 'a') as f:
        f.write('%s\n' % s)

model.eval()
with torch.set_grad_enabled(False):  # save memory during inference

    train_mae, train_mse, train_gender = compute_mae_and_mse(model, train_loader,
                                               device=DEVICE)
    valid_mae, valid_mse, valid_gender = compute_mae_and_mse(model, valid_loader,
                                               device=DEVICE)
    test_mae, test_mse, test_gender= compute_mae_and_mse(model, test_loader,
                                             device=DEVICE)

    s = 'MAE/RMSE/Gender: | Train: %.2f/%.2f/%.2f | Valid: %.2f/%.2f/%.2f | Test: %.2f/%.2f/%.2f' % (
        train_mae, torch.sqrt(train_mse),train_gender,
        valid_mae, torch.sqrt(valid_mse),valid_gender,
        test_mae, torch.sqrt(test_mse),test_gender)
    print(s)
    with open(LOGFILE, 'a') as f:
        f.write('%s\n' % s)

s = 'Total Training Time: %.2f min' % ((time.time() - start_time)/60)
print(s)
with open(LOGFILE, 'a') as f:
    f.write('%s\n' % s)

Epoch: 001/075 | Batch 0000/0156 | Cost: 50.6461
Epoch: 001/075 | Batch 0020/0156 | Cost: 16.4897
Epoch: 001/075 | Batch 0040/0156 | Cost: 13.0470
Epoch: 001/075 | Batch 0060/0156 | Cost: 11.1132
Epoch: 001/075 | Batch 0080/0156 | Cost: 10.9764
Epoch: 001/075 | Batch 0100/0156 | Cost: 9.8688
Epoch: 001/075 | Batch 0120/0156 | Cost: 9.7474
Epoch: 001/075 | Batch 0140/0156 | Cost: 10.5622
MAE/RMSE/Gender: | Current Valid: 5.18/6.53/87.12 Ep. 0 | Best Valid : 5.18/6.53/87.12 Ep. 0
Time elapsed: 2.41 min
Epoch: 002/075 | Batch 0000/0156 | Cost: 9.4017
Epoch: 002/075 | Batch 0020/0156 | Cost: 9.5267
Epoch: 002/075 | Batch 0040/0156 | Cost: 9.3737
Epoch: 002/075 | Batch 0060/0156 | Cost: 8.7123
Epoch: 002/075 | Batch 0080/0156 | Cost: 9.1114
Epoch: 002/075 | Batch 0100/0156 | Cost: 9.7678
Epoch: 002/075 | Batch 0120/0156 | Cost: 9.5226
Epoch: 002/075 | Batch 0140/0156 | Cost: 9.0396
MAE/RMSE/Gender: | Current Valid: 5.91/7.41/91.06 Ep. 1 | Best Valid : 5.18/6.53/87.12 Ep. 0
Time elapsed: 3.8

In [7]:
########## EVALUATE BEST MODEL ######
model.load_state_dict(torch.load(os.path.join(Path, 'Age_and_Gender.pt')))
model.eval()

with torch.set_grad_enabled(False):
    train_mae, train_mse, train_gender = compute_mae_and_mse(model, train_loader,
                                               device=DEVICE)
    valid_mae, valid_mse ,valid_gender= compute_mae_and_mse(model, valid_loader,
                                               device=DEVICE)
    test_mae, test_mse, test_gender = compute_mae_and_mse(model, test_loader,
                                             device=DEVICE)

    s = 'MAE/RMSE/Gender: | Best Train: %.2f/%.2f/%.2f | Best Valid: %.2f/%.2f/%.2f | Best Test: %.2f/%.2f/%.2f' % (
        train_mae, torch.sqrt(train_mse),train_gender,
        valid_mae, torch.sqrt(valid_mse),valid_gender,
        test_mae, torch.sqrt(test_mse), test_gender)
    print(s)
    with open(LOGFILE, 'a') as f:
        f.write('%s\n' % s)


########## SAVE PREDICTIONS ######
all_pred1 = []
all_pred2 = []
all_probas = []
with torch.set_grad_enabled(False):
    for batch_idx, (features, target1, target2, levels) in enumerate(test_loader):

        features = features.to(DEVICE)
        logits1,logits2,probas1,probas2 = model(features)
        all_probas.append(probas1)
        predict_levels = probas1 > 0.5
        predictions = probas2 > 0.5
        predicted_labels1 = torch.sum(predict_levels, dim=1)
        predicted_labels2 = torch.sum(predictions, dim=1)
        lst1 = [str(int(i)) for i in predicted_labels1]
        lst2 = [str(int(j)) for j in predicted_labels2]
        all_pred1.extend(lst1)
        all_pred2.extend(lst2)

torch.save(torch.cat(all_probas).to(torch.device('cpu')), TEST_ALLPROBAS)
with open(TEST_PREDICTIONS1, 'w') as f:
    all_pred1 = ','.join(all_pred1)
    f.write(all_pred1)
with open(TEST_PREDICTIONS2, 'w') as f:
    all_pred2 = ','.join(all_pred2)
    f.write(all_pred2)

MAE/RMSE/Gender: | Best Train: 0.81/1.20/99.40 | Best Valid: 2.67/3.73/98.56 | Best Test: 2.77/3.81/98.66
